In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.ensemble import RandomForestRegressor as RFR

from functools import partial
from bayes_opt import BayesianOptimization
import random

import tensorflow as tf
from keras.layers import (Dense,Conv1D,BatchNormalization,GlobalAveragePooling1D
                          ,MaxPooling1D,AveragePooling1D,Multiply,Add)

from tensorflow.keras.models import model_from_json
from define_CNN_class import MyResNetModel
from define_CNN_class import ResidualBlock
from define_CNN_class import NoiseGate

ニューラルネットワークを宣言

In [ ]:
custom_objects={
        'MyResNetModel': MyResNetModel,
        'ResidualBlock':ResidualBlock,
        'NoiseGate':NoiseGate        
        }

with open(r'cnn_model_structual_data.json', 'r') as json_file:

    json_string = json_file.read()
    model = model_from_json(json_string,custom_objects=custom_objects)


model.load_weights(r"cnn_model.weights.h5")
model.summary()
model.compile(optimizer='adam', loss='mse')


In [ ]:
df1= pd.read_excel(r"PKSP_Q84mse(25,40,55)v2.xlsx",sheet_name="解析用平均値",header=1)
df1_X=df1[df1['Temp'] == 298]
df1_X=df1_X[df1['サンプルNo.'] >=17]
NMR_columns=df1_X.columns[10:320]

NMR_data=df1_X.copy().loc[:,NMR_columns]

for i in range(len(NMR_data)):
    data=NMR_data.iloc[i,:].copy()
    NMR_data.iloc[i,:]= (data-np.min(data))/(np.max(data)-np.min(data))
    #Nomalized_data.iloc[i,:]= data/np.max(data)

    #print(np.max(Nomalized_data.iloc[i,:]))
    #print(np.min(Nomalized_data.iloc[i,:]))

process_columns=df1_X.columns[2:6]#成型条件が格納されているcolumnを取得
process_data=df1_X.copy().loc[:,process_columns]#columnをもとにNMRデータを取得

#print(NMR_data)

index=[]
for i in range(len(NMR_data)):
    index.append(f"Q{i+17}-MSE-298-1")#サンプル番号、パルス名、温度、N番号

NMR_data.index=index
#print(NMR_data)

physics_data=pd.read_csv(r"物性のデータ.csv").loc[:,"y1":]
physics_data.index=index
process_data.index=index
print(physics_data)
print(process_data)

for i in range(4):
    le=LabelEncoder()
    le.fit(process_data.iloc[:,i])
    process_data.iloc[:,i]=le.transform(process_data.iloc[:,i])
    print(np.max( process_data.iloc[:,i]))

print(process_data)


In [ ]:
pred=[]
soft_noise=[]

CNN_smooth=pd.DataFrame()
CNN_gap=pd.DataFrame()
ave_smooth=pd.DataFrame()

for i in range(len(NMR_data)):
    #data=model.predict(average_smoothing(np.array(selected_MAPE_data.loc[f"Q1{i}-mape-298-1",:]).reshape(1,311,1),num=10)).reshape(-1)
    pred=model.predict(np.array(NMR_data.iloc[i,:]).reshape(1,310,1))
                       
    data,gap=pred[0].reshape(-1),pred[1].reshape(-1)#出力を取得
    CNN_smooth[f"Q{i+17}-MSE-298-1"]=data
    CNN_gap[f"Q{i+17}-MSE-298-1"]=gap
    #サンプル番号、パルス名、温度、N番号をラベルにする

    
CNN_smooth=CNN_smooth.T
CNN_gap=CNN_gap.T

CNN_gap.columns=[f"gap_{i}" for i in range(len(CNN_gap.columns))]
#ave_smooth=ave_smooth.T

CNN_smooth.columns=columns=np.arange(310)*0.00064

print(CNN_smooth.shape)
print(CNN_smooth)


print(CNN_gap.shape)
print(CNN_gap)


In [ ]:
SS_X=StandardScaler()
ss_physics_data=pd.DataFrame(SS_X.fit_transform(physics_data),index=physics_data.index,columns=physics_data.columns)

SS_X=StandardScaler()
ss_CNN_gap=pd.DataFrame(SS_X.fit_transform(CNN_gap),index=CNN_gap.index,columns=CNN_gap.columns)


process_physics_gap=pd.concat([process_data,ss_physics_data,ss_CNN_gap],axis=1)
print(process_physics_gap)

In [ ]:
df1= pd.read_excel(r"PKSP_Q84mse(25,40,55)v2.xlsx",sheet_name="解析用平均値",header=1)
df1_X=df1[df1['Temp'] == 298]
df1_X=df1_X[df1['サンプルNo.'] >=17]
NMR_columns=df1_X.columns[10:320]#NMRデータが格納されているcolumnを取得
NMR_data=df1_X.copy().loc[:,NMR_columns]#columnをもとにNMRデータを取得

for i in range(len(NMR_data)):
    data=NMR_data.iloc[i,:].copy()#安全のため一行ずつコピーを生成
    CNN_smooth.iloc[i,:]=(CNN_smooth.iloc[i,:].copy()*((np.max(data)-np.min(data))))+np.min(data)#正規化されていたデータを元に戻す

print(CNN_smooth)


In [ ]:

print(np.array(CNN_gap.iloc[:,0]).reshape(-1).shape)
print(physics_data[f"y1"].shape)

SS_X=StandardScaler()
SS_X.fit((CNN_gap))
ss_gap=pd.DataFrame(SS_X.transform((CNN_gap)))


target_gap_list=[]
for i in range(1,10,1):
    print(f"y{i}")
    max_score=0
    index_num=0
    SS_Y=StandardScaler()
    SS_Y.fit(np.array(physics_data[f"y{i}"]).reshape(-1,1))
    y_data=SS_Y.transform(np.array(physics_data[f"y{i}"]).reshape(-1,1))
    
    num=64

    for j in range(256):
        score=np.corrcoef(np.array(ss_gap.iloc[0:num,j]).reshape(-1),y_data[0:num].reshape(-1))[0,1]
        if (score)>abs(max_score):
            max_score=(score)
            index_num=j
      
        #print(np.corrcoef(np.array(CNN_gap.iloc[:,j]).reshape(-1),y_data.reshape(-1)))

    print(index_num)
    print(max_score)

    target_gap_list.append(f"gap_{index_num}")

In [ ]:
SS_X=StandardScaler()
ss_physics_data=pd.DataFrame(SS_X.fit_transform(physics_data),index=physics_data.index,columns=physics_data.columns)

SS_X=StandardScaler()
ss_CNN_gap=pd.DataFrame(SS_X.fit_transform(CNN_gap),index=CNN_gap.index,columns=CNN_gap.columns)


process_physics_gap=pd.concat([process_data,ss_physics_data,ss_CNN_gap],axis=1)
print(process_physics_gap)

In [ ]:
print(process_physics_gap.iloc[:,:5].head())


# BaysianOptimizationで最適化する関数を定義する
# 結晶化温度/℃ ,結晶化時間 ,核材の濃度/％ ,冷結晶化の4要素を指定させる
#入力は任意の実数で関数内で離散的な実現値に変換する
#実現値でprocess_dataのDataFrameを探索して行名or行番号を取得
#取得した行番号で物性値 or GAPのDataFrameを検索してreturn
def pick_up(crystallize_temp ,crystallize_length ,per_nucleating_agent ,cold_crystallize,process_physics_gap ,search_columns):

    crystallize_temp = int(crystallize_temp)#実数で渡された値を離散化
    crystallize_length = int(crystallize_length)
    per_nucleating_agent = int(per_nucleating_agent)
    cold_crystallize = int(cold_crystallize)
    
    data = process_physics_gap.set_index(['結晶化温度/℃', '結晶化時間', '核材の濃度/％', '冷結晶化'])#カラム名を複合キーに指定する
    key=(crystallize_temp,crystallize_length,per_nucleating_agent,cold_crystallize)#関数に渡された値で検索をかける
    
    if key in data.index:
        result = data.loc[key,f"{search_columns}"]
    else:
        result = 0 #複合キーに網羅性がないため、実現しない値での検索が起こった場合の例外処理として0を返すようにする

    return result


rocked_pick_up = partial(pick_up,process_physics_gap=process_physics_gap,search_columns="y1")
#一部の変数を固定して代入できるようにする

#print(rocked_pick_up(1,1,1,0))
print(rocked_pick_up(crystallize_temp=1 ,crystallize_length=1 ,per_nucleating_agent=1 ,cold_crystallize=0))



df_target_names=pd.DataFrame()
df_target_names["y"]=[f"y{i}" for i in range(1,10,1)]
df_target_names["gap"]=target_gap_list
df_target_names.index=range(1,10,1)
print(df_target_names)

In [ ]:

df_target_names=pd.DataFrame()
df_target_names["y"]=[f"y{i}" for i in range(1,10,1)]
df_target_names["gap"]=target_gap_list
df_target_names.index=range(1,10,1)
print(df_target_names)


# BaysianOptimizationで最適化する関数を定義する
# 結晶化温度/℃ ,結晶化時間 ,核材の濃度/％ ,冷結晶化の4要素を指定させる
#入力は任意の実数で関数内で離散的な実現値に変換する
#実現値でprocess_dataのDataFrameを探索して行名or行番号を取得
#取得した行番号で物性値 or GAPのDataFrameを検索してreturn
#def pick_up(crystallize_temp ,crystallize_length ,per_nucleating_agent ,cold_crystallize,process_physics_gap ,search_columns):
def pick_up(crystallize_temp ,crystallize_length ,per_nucleating_agent ,process_physics_gap ,search_columns):

    crystallize_temp = int(crystallize_temp)#実数で渡された値を離散化
    crystallize_length = int(crystallize_length)
    per_nucleating_agent = int(per_nucleating_agent)
    #cold_crystallize = int(cold_crystallize)
    cold_crystallize=0 #冷結晶化のデータを除外するかどうかは要検討

    data = process_physics_gap.set_index(['結晶化温度/℃', '結晶化時間', '核材の濃度/％', '冷結晶化'])#カラム名を複合キーに指定する
    key=(crystallize_temp,crystallize_length,per_nucleating_agent,cold_crystallize)#関数に渡された値で検索をかける
    
    if key in data.index:
        result = data.loc[key,f"{search_columns}"]
    else:
        result = 0 #複合キーに網羅性がないため、実現しない値での検索が起こった場合の例外処理として0を返すようにする

    return result


df_result_list=[]#各回の試行の結果をまとめたDataFrameを保存する配列

for k in range(200):
    print(f"{k}周目")
    df_result=pd.DataFrame()
    random_state=42 + k #疑似乱数のシード値にkを足して各実験の初回探索が変わるようにする

    #物性値とGAPで網羅的に探索
    for i in range(1,len(df_target_names)+1,1):# 9行分
        for j  in df_target_names.columns: #2列分(yiとgap)を探索する

            target=df_target_names.loc[i,j]
            rocked_pick_up = partial(pick_up, process_physics_gap = process_physics_gap, search_columns = target)
            #一部の変数を固定して代入できるようにする


            #ベイズ最適化で探索するパラメータ空間を定義する
            process_bo = BayesianOptimization(
                f=rocked_pick_up,  #最適化する関数
                pbounds={
                    'crystallize_temp': (-0.5, 0.5+np.max(process_physics_gap["結晶化温度/℃"])),#int型で離散化するため、探索対象の最大値＋0.99の値域を与えてほぼ均等に網羅性を確保
                    'crystallize_length': (-0.5, 0.5+np.max(process_physics_gap["結晶化時間"])),
                    'per_nucleating_agent': (-0.5, 0.5+np.max(process_physics_gap["核材の濃度/％"]))
                },
                random_state=random_state #初回探索の対象を決める疑似乱数のシード値
                )

            #ベイズ最適化を実行（scoreが最大となるようにパラメータを探索していく）


            process_bo.maximize(init_points=20, n_iter=100)

            #試行した全課程の情報を取得
            all_trials = process_bo.res
            print(all_trials)

            trials_df = pd.DataFrame([{
                **trial['params'],   # 各試行のパラメータ
                f'{target}': trial['target']  # 試行結果のスコア
            } for trial in all_trials])


            print(trials_df)
            if "y" in target:
                df_result[target]=trials_df[target]
                
            elif "gap" in target:
                df_result[f"y{i}_{target}"]=[pick_up(**trial['params'],process_physics_gap=process_physics_gap,search_columns=f"y{i}") for trial in all_trials]
            
            print(df_result)
            
    df_result_list.append(df_result)
        
#実際に調べた有効なサンプル数を表示する

print("y1")
print(np.max(df_result_list[0]["y1"]))
print(np.max(df_result_list[0]["y1_gap_240"]))

print("y2")
print(np.max(df_result_list[0]["y2"]))
print(np.max(df_result_list[0]["y2_gap_113"]))

print("y3")
print(np.max(df_result_list[0]["y3"]))
print(np.max(df_result_list[0]["y3_gap_240"]))

print("y4")
print(np.max(df_result_list[0]["y4"]))
print(np.max(df_result_list[0]["y4_gap_240"]))

print("y5")
print(np.max(df_result_list[0]["y5"]))
print(np.max(df_result_list[0]["y5_gap_178"]))

In [ ]:
for i  in range(1,10,1):
    print(np.max(process_physics_gap[f"y{i}"]))

In [ ]:
r2_score_list=[]
#物性ごとに実験の平均値を表示
for i in range(9):#物性の数
    df_random_search=pd.DataFrame()

    #空のDataFrameを宣言
    df_bo_improve_max_y=pd.DataFrame()
    df_bo_improve_max_gap=pd.DataFrame()
    for j in range(200):#実験回数
       

        random_search=[pick_up(crystallize_temp=random.uniform(-0.5, 0.5+np.max(process_physics_gap["結晶化温度/℃"])) ,crystallize_length=random.uniform(-0.5, 0.5+np.max(process_physics_gap["結晶化時間"])) ,per_nucleating_agent=random.uniform(-0.5, 0.5+np.max(process_physics_gap["核材の濃度/％"])) ,process_physics_gap=process_physics_gap ,search_columns=f"y{i+1}") for _ in range(120)]
        df_random_search[j]=[np.max(random_search[:k+1]) for k in range(120)]



        #それぞれの実験のmaxの推移を表現した配列をリスト内包表記で作成
        df_bo_improve_max_y[j]=[np.max(df_result_list[j].iloc[:k+1,i*2]) for k in range(len(df_result))]
        df_bo_improve_max_gap[j]=[np.max(df_result_list[j].iloc[:k+1,i*2+1]) for k in range(len(df_result))]

    #リス他内包表記で行ごとの平均値を配列に変換
    print(f"y{i+1}")
    #print(np.max(df_result_list.iloc[:,i*2]))
    #print(np.max(df_result_list.iloc[:,i*2+1]))

    
    plt.plot(range(len(df_result)),[np.mean(df_bo_improve_max_y.iloc[j,:]) for j in range(len(df_result))],label="y")
    plt.plot(range(len(df_result)),[np.mean(df_bo_improve_max_gap.iloc[j,:]) for j in range(len(df_result))],label="gap")
    plt.plot(range(len(df_result)),[np.mean(df_random_search.iloc[j,:]) for j in range(len(df_result))],label="random")

    plt.plot(range(len(df_result)),[np.max(process_physics_gap[f"y{i+1}"]) for j in range(len(df_result))],label="max",color="black")

    plt.legend()
    plt.show()
    print("gap",r2_score([np.mean(df_bo_improve_max_y.iloc[j,:]) for j in range(len(df_result))],[np.mean(df_bo_improve_max_gap.iloc[j,:]) for j in range(len(df_result))]))
    print("random:",r2_score([np.mean(df_bo_improve_max_y.iloc[j,:]) for j in range(len(df_result))],[np.mean(df_random_search.iloc[j,:]) for j in range(len(df_result))]))

    r2_score_list.append(r2_score([np.mean(df_bo_improve_max_y.iloc[j,:]) for j in range(len(df_result))],[np.mean(df_bo_improve_max_gap.iloc[j,:]) for j in range(len(df_result))]))

plt.bar(range(len(r2_score_list)),r2_score_list)